In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib import font_manager, rc
import platform

# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [5]:
def softmax(x) :
    c = np.max(x, axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [19]:
x= np.array([[-1,0],[1,2],[2,0]])
x

array([[-1,  0],
       [ 1,  2],
       [ 2,  0]])

In [71]:
def categorical_crossentropy(t,y):
    return np.mean(-t * np.log(y))

In [86]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
class Sigmoid:
    def __init__(self):
        pass
    
    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out
    
    def backward(self, dout):
        dx = ((1-self.out)*self.out) * dout
        return dx
    
class Affine:
    def __init__(self,w,b):
        self.w = w
        self.b = b
        self.x = None
        self.origin_shape = None
        self.dw = None
        self.db = None
    
    def forward(self,x):
        self.origin_shape = x.shape
        self.x = x
        out = np.dot(x,self.w) + self.b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.w.T)
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        dx = dx.reshape(self.origin_shape)
        return dx, self.dw, self.db
    
class Loss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        self.predict = None
        
    def forward(self,y,t):
        self.y = y
        self.t = t
        self.loss = categorical_crossentropy(self.t, self.y)
        return self.loss
    
    def backward(self,dout = 1):
        dx = self.y - self.t
        return dx

In [7]:
xx = Sigmoid()

In [9]:
a = xx.forward(np.random.randn(2,3))

In [10]:
xx.backward(a)

array([[0.09361349, 0.03932356, 0.14783214],
       [0.03392713, 0.14351435, 0.12609638]])

In [26]:
y = np.array([[7,6],[-2,5],[4,1]])

In [24]:
relu_d = Relu()
relu_d.forward(x)

array([[0, 0],
       [1, 2],
       [2, 0]])

In [27]:
relu_d.backward(y)

array([[ 0,  0],
       [-2,  5],
       [ 4,  0]])

In [59]:
sig = sigmoid

In [60]:
sig(np.random.randn(2,3))

array([[0.38436498, 0.23262633, 0.12994354],
       [0.17693802, 0.07657477, 0.7489652 ]])

In [66]:
Affine1 = Affine(np.random.randn(5,3),np.zeros(3))

In [67]:
Affine1.forward(np.random.randn(10,5))

array([[-0.53444476,  0.70366698, -0.48281884],
       [ 2.29484412, -0.94666759, -3.39876404],
       [-2.96643611, -2.04356294,  0.43939576],
       [-2.3043202 , -1.08764593,  1.8439517 ],
       [-0.01428214,  0.35715397, -0.04761793],
       [-4.72800148,  3.20538883, -1.61072392],
       [-5.20584864,  4.43128573,  2.08908369],
       [-3.81295376,  0.52898543,  3.24114587],
       [ 1.7538301 , -1.69012153,  1.25380734],
       [ 0.42070439,  2.50489234,  0.93272485]])

In [74]:
Affine1.backward(np.random.randn(10,3))

(array([[ 0.39576501, -0.88663313,  2.7140866 , -0.13500568,  0.25818474],
        [ 0.64622149, -0.2370506 ,  0.34020653,  0.57636187,  1.32554666],
        [ 0.66539526, -2.1356573 ,  6.46526248, -0.57018593, -0.23909205],
        [-1.47388092,  1.37193909, -1.50478162, -1.37668533, -0.96647668],
        [-1.75134978,  0.06245018,  0.42017854, -1.76279826, -4.45198249],
        [ 0.6435247 , -0.21557712, -1.02561586,  0.96672063,  0.44127059],
        [ 1.51420921, -1.78192978,  0.68567704,  1.78975041, -0.74786511],
        [-2.33682962,  1.70960003,  0.06040083, -2.74473381, -1.27755681],
        [-0.30442506,  1.35565742, -5.06891411,  0.75355361, -0.18199585],
        [ 1.18113924, -0.213838  , -3.51276899,  2.19148968,  0.24448657]]),
 array([[ 2.05647447,  4.19118745, -2.00504628],
        [-3.50711108, -0.37284952, -3.91425713],
        [-1.32877853,  2.73678766, -1.00140948],
        [ 1.22712266, -0.56321925, -2.42913131],
        [-3.98743891, -1.20390859, -5.34872602]]),
 

In [76]:
y = softmax(np.random.randn(10,3))
y

array([[0.01625139, 0.68478362, 0.29896499],
       [0.38766205, 0.34178007, 0.27055788],
       [0.04979505, 0.22035042, 0.72985453],
       [0.4308588 , 0.41306679, 0.15607442],
       [0.09521766, 0.42725617, 0.47752617],
       [0.11908561, 0.56504533, 0.31586906],
       [0.44452981, 0.02124384, 0.53422634],
       [0.23815235, 0.4383816 , 0.32346605],
       [0.49500368, 0.26280615, 0.24219017],
       [0.17815059, 0.20700576, 0.61484365]])

In [105]:
x = softmax(np.random.randn(10,3))

In [110]:
t = np.random.randn(10,3)

In [111]:
x = np.argmax(t,axis=1)

In [114]:
t = np.zeros((10,3))

In [115]:
for i, k in enumerate(x):
    t[i,k] = k

In [116]:
t

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 2.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [117]:
loss = Loss()

In [118]:
loss.forward(y,t)

0.14624756944695763